## Matrix Factorization
*ref: http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/


In this exercise, we will implement matrix factorization using stochastic gradient descent.

Recall from the lecture, the error of entry i,j is given by 
$$e_{ij} = r_{ij} - p{ij}$$
$$e_{ij}^2 = (r_{ij} - p_{ij})^2 = (r_{ij} - \sum_{k=1}^K u_{ik}m_{kj})^2$$

Now the partial derivatives of the square error,
$$\frac{\partial e_{ij}^2}{\partial u_{ik}} = -2(r_{ij} - p_{ij})(m_{kj}) = -2e_{ij}m_{kj}$$
$$\frac{\partial e_{ij}^2}{\partial m_{kj}} = -2(r_{ij} - p_{ij})(u_{ik}) = -2e_{ij}u_{ik}$$

The update rules are:
$$u_{ik}^{t+1} = u_{ik}^{t} - \gamma \frac{\partial e_{ij}^2}{\partial u_{ik}} = u_{ik}^{t} + 2 \gamma e_{ij}m_{kj}$$
$$m_{kj}^{t+1} = m_{kj}^{t} - \gamma \frac{\partial e_{ij}^2}{\partial m_{kj}} = m_{kj}^{t} + 2 \gamma e_{ij}u_{ik}$$

In addition, we will introduce a regularization term to the error term to restrict the norm of the U and M.
$$e_{ij}^2 = (r_{ij} - \sum_{k=1}^K u_{ik}m_{kj})^2 + \frac{\beta}{2}\sum_{k=1}^K(|u_{k}|^2 + |m_{k}|^2)$$

The update rules now become.
$$u_{ik}^{t+1} = u_{ik}^{t} + \gamma(2*e_{ij}m_{kj} - \beta u_{ij})$$
$$m_{kj}^{t+1} =  m_{kj}^{t} + \gamma(2*e_{ij}u_{ik} - \beta m_{kj})$$



In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def matrix_factorization(R, U, M, K, max_iter=7000, learning_rate=1e-4, beta=0.02, tol=1e-3):
    M = M.T
    x,y = np.nonzero(R)
    for it in range(max_iter):
        # create a random ordering
        order = np.random.permutation(len(x))
        for step in range(len(order)):
            i = x[order[step]]
            j = y[order[step]]
            eij = R[i,j] - np.dot(U[i,:],M[:,j])
            for k in range(K):
                U[i,k] = U[i,k] + learning_rate*(2*eij*M[k,j] - beta*U[i,k])
                M[k,j] = M[k,j] + learning_rate*(2*eij*U[i,k] - beta*M[k,j])
        # calculate the error
        P = np.dot(U,M)
        e2 = 0
        for step in range(len(order)):
            i = x[order[step]]
            j = y[order[step]]
            e2 += pow(R[i,j] - P[i,j],2)
            for k in range(K):
                e2 += beta/2.0 * (pow(U[i,k],2) + pow(M[k,j],2))
        if e2 < tol:
            break
    return U,M.T   

In [4]:
R = [[5,3,0,1],
     [4,0,0,1],
     [1,1,0,5],
     [1,0,0,4],
     [0,1,5,4]];

R = np.array(R)
I,J = R.shape
K = 2
U = np.random.rand(I,K)
M = np.random.rand(J,K)
nU, nM = matrix_factorization(R,U,M,K)
P = np.dot(nU,nM.T)

In [5]:
P

array([[ 5.00250874,  2.92915433,  3.91568909,  0.99141214],
       [ 3.95904596,  2.32746707,  3.29765012,  0.99747303],
       [ 1.06401983,  0.83021077,  5.26066751,  4.95353557],
       [ 0.97502083,  0.73517857,  4.27376041,  3.95343036],
       [ 1.80434345,  1.21888432,  4.88244486,  4.07444695]])

In [6]:
U

array([[ 2.18163441, -0.08272148],
       [ 1.7354388 ,  0.02925377],
       [ 0.66159105,  2.09286862],
       [ 0.58185161,  1.65714713],
       [ 0.94172045,  1.62414689]])

In [7]:
M

array([[ 2.28489943, -0.21389264],
       [ 1.34160256, -0.02741762],
       [ 1.86776377,  1.92318413],
       [ 0.53773494,  2.19687701]])